In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Fraud.csv')

In [ ]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
data.drop(['nameOrig','nameDest'],axis=1,inplace=True)
def isuni(df):
  for col in df:
      print(f'{col}:{df[col].unique()}')

data.head()
def isnul(df):
  for col in df:
      print(f'{col}:{df[col].isnull().unique()}')

isnul(data)

step:[False]
type:[False]
amount:[False]
oldbalanceOrg:[False]
newbalanceOrig:[False]
oldbalanceDest:[False]
newbalanceDest:[False]
isFraud:[False]
isFlaggedFraud:[False]


In [ ]:
isuni(data)

In [ ]:
data.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [ ]:
data['isFraud'].value_counts()

0    6354407
1       8213
Name: isFraud, dtype: int64

In [ ]:
data['type'].value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [ ]:
df2=pd.get_dummies(data=data,columns=['type'])

In [ ]:
df2.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,1,0,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,0,1,0


In [ ]:
cols_to_scale=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','step']
df2[cols_to_scale]=scaler.fit_transform(df2[cols_to_scale])

In [ ]:
isuni(df2)

In [ ]:
df2.dtypes
X=df2.drop('isFraud',axis='columns')
Y=df2.isFraud
print(X.shape,Y.shape)

(6362620, 12) (6362620,)


In [ ]:
df_majority = df2[df2.isFraud == 0]
df_minority = df2[df2.isFraud == 1]

In [ ]:
# sample_size = len(df_minority)
# df_majority_downsampled = df_majority.sample(n=sample_size, random_state=42)
# df_balanced = pd.concat([df_majority_downsampled, df_minority])
# X_balanced = df_balanced.drop('isFraud', axis='columns')
# y_balanced = df_balanced.isFraud
# xtrain_balanced, xtest_balanced, ytrain_balanced, ytest_balanced = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=256)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3,random_state=256)

In [ ]:
xtrain_resampled, ytrain_resampled = sm.fit_resample(xtrain, ytrain)
# xtrain_resampled, ytrain_resampled = sm.fit_resample(xtrain_balanced, ytrain_balanced)

In [ ]:
# # from sklearn.pipeline import make_pipeline
# # from sklearn.svm import SVC
# # clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# xtrain_resampled, ytrain_resampled = make_classification(n_samples=1000, n_features=12,random_state=0, shuffle=False)
# clf = RandomForestClassifier(max_depth=2, random_state=0)
# clf.fit(xtrain_resampled, ytrain_resampled)
# # print(clf.predict([[0, 0, 0, 0]]))
# # clf.fit(xtrain_resampled, ytrain_resampled)
xtrain_resampled = scaler.fit_transform(xtrain_resampled)
xtest_scaled = scaler.transform(xtest)


In [ ]:
# ypred=clf.predict(xtest)

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix

# print("Confusion Matrix:")
# print(confusion_matrix(ytest,ypred))

# print("\nClassification Report:")
# print(classification_report(ytest,ypred))

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(xtrain_resampled.shape[1],)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    xtrain_resampled, ytrain_resampled,
    epochs=50,
    batch_size=512,
    validation_split=0.1,
    callbacks=[early_stopping]
)

# model.fit(xtrain_resampled, ytrain_resampled, epochs=10, batch_size=512, validation_split=0.1)


Epoch 1/50
15638/15638 [==============================] - 71s 4ms/step - loss: 0.0737 - accuracy: 0.9706 - val_loss: 0.0336 - val_accuracy: 0.9927
Epoch 2/50
15638/15638 [==============================] - 69s 4ms/step - loss: 0.0592 - accuracy: 0.9771 - val_loss: 0.0423 - val_accuracy: 0.9874
Epoch 3/50
15638/15638 [==============================] - 65s 4ms/step - loss: 0.0569 - accuracy: 0.9782 - val_loss: 0.0471 - val_accuracy: 0.9850
Epoch 4/50
15638/15638 [==============================] - 67s 4ms/step - loss: 0.0555 - accuracy: 0.9790 - val_loss: 0.0497 - val_accuracy: 0.9861
Epoch 5/50
15638/15638 [==============================] - 65s 4ms/step - loss: 0.0530 - accuracy: 0.9803 - val_loss: 0.0294 - val_accuracy: 0.9926
Epoch 6/50
15638/15638 [==============================] - 64s 4ms/step - loss: 0.0514 - accuracy: 0.9810 - val_loss: 0.0339 - val_accuracy: 0.9926
Epoch 7/50
15638/15638 [==============================] - 66s 4ms/step - loss: 0.0506 - accuracy: 0.9815 - val_loss: 0

In [ ]:
test_loss, test_acc = model.evaluate(xtest_scaled, ytest)
print("Test Accuracy:", test_acc)


59650/59650 [==============================] - 144s 2ms/step - loss: 0.0439 - accuracy: 0.9834
Test Accuracy: 0.9834104180335999


In [ ]:
ypred = model.predict(xtest_scaled)


59650/59650 [==============================] - 97s 2ms/step


In [ ]:
ypred_binary = (ypred > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:")
print(confusion_matrix(ytest,ypred_binary))

print("\nClassification Report:")
print(classification_report(ytest,ypred_binary))

Confusion Matrix:
[[1874675   31640]
 [     26    2445]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99   1906315
           1       0.07      0.99      0.13      2471

    accuracy                           0.98   1908786
   macro avg       0.54      0.99      0.56   1908786
weighted avg       1.00      0.98      0.99   1908786



In [ ]:
# Confusion Matrix Custom ANN:
# [[1881682   24633]
#  [     16    2455]]

# Classification Report:
#               precision    recall  f1-score   support

#            0       1.00      0.99      0.99   1906315
#            1       0.09      0.99      0.17      2471

#     accuracy                           0.99   1908786
#    macro avg       0.55      0.99      0.58   1908786
# weighted avg       1.00      0.99      0.99   1908786


# MAX achieved so far on custom ANN


In [ ]:
import xgboost as xgb
model2 = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Reduce batch size and implement early stopping
batch_size = 32
early_stopping_rounds = 10

# Train the model
model2.fit(
    xtrain_resampled, ytrain_resampled,
    eval_set=[(xtest_scaled, ytest)],
    eval_metric='logloss',
    early_stopping_rounds=early_stopping_rounds,
    verbose=True
)


In [ ]:
ypredxgb = model2.predict(xtest_scaled)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:")
print(confusion_matrix(ytest,ypredxgb))

print("\nClassification Report:")
print(classification_report(ytest,ypredxgb))

Confusion Matrix:
[[1895827   10488]
 [     14    2457]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00   1906315
           1       0.19      0.99      0.32      2471

    accuracy                           0.99   1908786
   macro avg       0.59      0.99      0.66   1908786
weighted avg       1.00      0.99      1.00   1908786



In [ ]:
# Confusion Matrix XGBoost:
# [[1895827   10488]
#  [     14    2457]]

# Classification Report:
#               precision    recall  f1-score   support

#            0       1.00      0.99      1.00   1906315
#            1       0.19      0.99      0.32      2471

#     accuracy                           0.99   1908786
#    macro avg       0.59      0.99      0.66   1908786
# weighted avg       1.00      0.99      1.00   1908786

Data Cleaning:

Missing Values: There is no mention of missing values in the dataset, so it appears that the dataset didn't have any missing values to clean.

Multicollinearity: The presence of multicollinearity was not discussed explicitly. Multicollinearity refers to high correlations between predictor variables, which can lead to unstable coefficients and interpretation issues.
Fraud Detection Model:

The fraud detection model discussed here is built using the XGBoost algorithm, a powerful gradient boosting technique. The model is trained to predict whether a transaction is fraudulent (class 1) or not (class 0) based on various features such as transaction type, amount, customer balance, etc.
Variable Selection:

Variables were selected based on domain knowledge and the available features in the dataset. Key features related to transactions, customer information, and balances were included in the model. Feature engineering techniques were applied, such as creating dummy variables for categorical features and scaling numerical features.
Performance Demonstration:

The model's performance is evaluated using the confusion matrix and classification report provided. These tools provide insights into the model's accuracy, precision, recall, and F1-score for both classes.
Key Predictive Factors:

The key factors that predict a fraudulent customer, based on the provided model, include high recall for class 1 (fraudulent transactions). The model is able to correctly identify a substantial portion of actual fraudulent transactions.
Interpretation of Factors:

Yes, the factors identified in the model make sense. The model's ability to achieve a high recall for class 1 suggests that it is effective at identifying fraudulent transactions. High recall is important in fraud detection because missing a fraudulent transaction can be more costly than having some false positives.
Prevention Measures during Infrastructure Update:

The company should adopt proactive measures while updating its infrastructure:
Implement advanced anomaly detection algorithms to identify unusual patterns.
Enhance real-time monitoring to flag suspicious activities.
Implement two-factor authentication and identity verification for high-value transactions.
Establish rules and alerts for transactions involving high-risk countries or unusual patterns.
Evaluating Implemented Actions:

After implementing the prevention measures, their effectiveness can be determined by monitoring key metrics:
Evaluate changes in the number of detected fraudulent transactions and successful prevention outcomes.
Analyze precision, recall, F1-score, and accuracy of the model on a regular basis.
Conduct regular audits and reviews to assess whether the measures are preventing new fraudulent tactics effectively.